### 분석 데이터 배경
- 데이터셋은 영화 27278개의 20000263 레이팅과 465564의 태깅으로 구성
- 1995년부터 2015년까지 138493의 유저들이 입력한 데이터로 구성
- 각 유저들은 최소 20개의 영화에 대한 레이팅을 진행했음

### 분석 데이터 설명
- 개인 정보 이슈로 인해서 개인 데모 정보는 포함되어 있지 않음
- 모든 유저들은 id로 표현이 되며 그 외의 정보는 제공되지 않음

### 데이터는 총 6가지 테이블로 구성

1. tag.csv: 유저들이 영화마다 적용한 태그 정보

- userId, movieId, tag, timestamp

2. rating.csv : 유저들이 매겨놓은 영화 레이팅 정보

- userId, movieId, rating, timestamp

3. movie.csv: 영화 정보

- movieId, title, genres

4. link.csv: 다른 정보들과 연계해서 분석할 수 있는 매핑 테이블

- movieId, imdbId, tmbdId

5. genome_scores.csv: 태그 정보와 관련된 정보가 들어있는 테이블

- movieId, tagId, relevance

6. genome_tags.csv: 태그 정보 매핑 테이블

- tagId, tag

##### 분석 참고 링크: https://www.kaggle.com/anmol4210/recommendation-system-collaborative-filtering

### 1. 패키지, 데이터 셋 호출

In [2]:
## 패키지 호출
import pandas as pd
import numpy as np

In [10]:
## 데이터 경로 지정
path= 'data/'
movies= pd.read_csv(path+'movie.csv')
ratings= pd.read_csv(path+'rating.csv')
genome_scores= pd.read_csv(path+'genome_scores.csv')
link= pd.read_csv(path+'link.csv')
genome_tag= pd.read_csv(path+'genome_tags.csv')

### 2. EDA

In [15]:
## 전체 데이터 살펴보기
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [29]:
len(movies)

27278

In [16]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [30]:
len(ratings)

20000263

In [19]:
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [18]:
link.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [20]:
genome_tag.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


### 2. 데이터 정제

In [55]:
## 각 데이터 내 불필요 컬럼 제거 후, movie와 rating 데이터 합치기
movie_dt= movies.drop(['genres'],axis=1)
rating_dt= ratings.drop(['timestamp'],axis=1)

dt= pd.merge(movie_dt,rating_dt,on=['movieId'])

In [56]:
## 데이터의 로우 제한 (overflow 때문)
dt = dt.iloc[:1000000,:]

In [57]:
## 데이터 피보팅
pivot_table= dt.pivot_table(index=['userId'],columns=['title'],values='rating')
pivot_table.head()

title,Ace Ventura: When Nature Calls (1995),Across the Sea of Time (1995),"Amazing Panda Adventure, The (1995)","American President, The (1995)",Angela (1995),Angels and Insects (1995),Anne Frank Remembered (1995),Antonia's Line (Antonia) (1995),Assassins (1995),Babe (1995),...,Unforgettable (1996),Up Close and Personal (1996),"Usual Suspects, The (1995)",Vampire in Brooklyn (1995),Waiting to Exhale (1995),When Night Is Falling (1995),"White Balloon, The (Badkonake sefid) (1995)",White Squall (1996),Wings of Courage (1995),"Young Poisoner's Handbook, The (1995)"
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
movie_watched = pivot_table["Bad Boys (1995)"]
similarity_with_other_movies = pivot_table.corrwith(movie_watched)  # find correlation between "Bad Boys (1995)" and other movies
similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=False)
similarity_with_other_movies.head()

title
Bad Boys (1995)                        1.000000
Headless Body in Topless Bar (1995)    0.723747
Last Summer in the Hamptons (1995)     0.607554
Two Bits (1995)                        0.507008
Shadows (Cienie) (1988)                0.494186
dtype: float64